In [3]:
import sys
sys.path.append('../')
import utils
import models
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import tensorflow as tf 
 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

model, x_test, y_test = models.train(epochs=10, batch_size=4)

1 Physical GPUs, 1 Logical GPUs
-------------------------------------
num of train data =  251
num of val data =  251
batch size =  4
epochs =  10
-------------------------------------
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d

In [15]:

index = 0 
x_text_img = cv2.imread(x_test[index]) / 255.0    
y_test_img = cv2.imread(y_test[index]) / 255.0
x_text_img = np.expand_dims(x_text_img, axis=0)

result = model.predict(x_text_img)
result_rgb = result.squeeze()[:,:,::-1]
x_test_img_rgb = x_text_img.squeeze()[:,:,::-1]
y_test_img_rgb = y_test_img[:,:,::-1]

plt.figure() 
fig, axs = plt.subplots(1, 3,figsize = (20,13))
axs[0].imshow(x_test_img_rgb)
axs[0].set_title('input')
axs[1].imshow(result_rgb)
axs[1].set_title('result')
axs[2].imshow(y_test_img_rgb)
axs[2].set_title('gt')

ResourceExhaustedError:  OOM when allocating tensor with shape[1,16,2592,3872] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node conv2d_1/convolution (defined at C:\Users\IT2-329\Anaconda3\envs\keras\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_18431]

Function call stack:
keras_scratch_graph


In [11]:
cv2.imwrite('input.bmp', np.uint8(x_test_img_rgb[:,:,::-1] * 255))
cv2.imwrite('gt.bmp', np.uint8(y_test_img_rgb[:,:,::-1] * 255))
cv2.imwrite('result.bmp', np.uint8(result_rgb[:,:,::-1] * 255))

True

In [18]:
model.save_weights('simple_ae_100.h5')